<a href="https://colab.research.google.com/github/joelkabamba/Python-Machine-Learning/blob/master/Copie_de_Bank_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap
!wget "https://drive.google.com/uc?export=download&id=1OMjEx6D_mDSJSb9-fgOfwD19rz0sZ4ra" -O dataset.csv

In [ ]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

import IPython

IPython.get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)

In [ ]:
import numpy as np
import pandas as pds
import seaborn as sns
import shap
import matplotlib
import matplotlib.pyplot as plt #visualization
%matplotlib inline

plt.rcParams["figure.figsize"] = (16,11)
font = {'weight' : 'normal',
        'size'   : 16}
matplotlib.rc('font', **font)

import itertools
import warnings
warnings.filterwarnings("ignore")
import os
import io
import plotly.offline as py #visualization
py.init_notebook_mode(connected=True) #visualization
import plotly.graph_objs as go #visualization
import plotly.tools as tls #visualization
import plotly.figure_factory as ff #visualization

In [ ]:
data = pd.read_csv('dataset.csv')
data.head()

In [ ]:
clean_column_name = []
columns = data.columns
for i in range(len(columns)):
    clean_column_name.append(columns[i].lower())
data.columns = clean_column_name

In [ ]:
data = data.drop(["rownumber", "customerid", "surname"], axis=1)

In [ ]:
print(data.shape)
data.head()

In [ ]:
np.sum(data.isna())

# Analyse exploratoire et Data Visualization

L'objectif de cette partie est de répondre à des questions que l'on se pose dans le but d'acquérir de la connaissance sur les données. Pour mener à bien cette partie, il faut se mettre dans la tête d'une personne qui ne connaît ni la situation, ni les données et qui poserait des questions essentielles.

## Analyse univariée

Effectuons une première visualisation de l'état du compte bancaire au moment de la récupération du jeu de données.

In [ ]:
sns.distplot(data['balance'])

Nous observons une forte proportion d'individus ayant un compte bancaire à 0€ le jour de la récupération des informations. Pour autant, cela signifie-t-il qu'il y a eu autant d'individus ayant exactement 0€ sur leur compte bancaire ce jour-là ?

In [ ]:
sns.distplot(data['balance'][data['balance'] > 0])

## Analyse multivariée

In [ ]:
sns.distplot(data.loc[data['exited'] == 1, 'age'], label="Churn")
sns.distplot(data.loc[data['exited'] == 0, 'age'], label="Non churn")
plt.legend()

In [ ]:
sns.boxplot(x='numofproducts', y='age', data=data)

In [ ]:
sns.violinplot(x='numofproducts', y='age', data=data)

### Analyse de la variable réponse

In [ ]:
data['exited'].value_counts().plot.pie(autopct=lambda x: '{:2.1f}%'.format(x), explode=[0, 0.1])

In [ ]:
sns.boxplot(x='numofproducts', y='age', data=data, hue="exited")

In [ ]:
data[(data['exited'] == 1) & (data['numofproducts'] == 4)].shape

# Nettoyage

In [ ]:
cleaned_data = data.copy()
cleaned_data = cleaned_data[~((cleaned_data['exited'] == 1) & (cleaned_data['numofproducts'] == 4))]
cleaned_data.shape

# Encodage

In [ ]:
X = cleaned_data.iloc[:, :-1].copy()
y = cleaned_data['exited']
X.head()

## Encodage binaire

In [ ]:
X['gender'] = data['gender'].apply(lambda x: 1 if x == "Female" else 0)
X.head()

In [ ]:
X = X.join(pd.get_dummies(data['geography']))
X.head()

In [ ]:
del X['geography']
X.head()

In [ ]:
clean_column_name = []
columns = X.columns
for i in range(len(columns)):
    clean_column_name.append(columns[i].lower())
X.columns = clean_column_name
X.head()

Avant d'entraîner un modèle, il faut s'assurer que l'on sépare bien le jeu de données en deux ensembles :

- Un ensemble de train
- Un ensemble de test

Il est essentiel de séparer le jeu de données en deux : on veut se rapprocher au maximum du séparateur en noir, pas en vert.

<center>
  <img width="400" src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/Overfitting.svg/1200px-Overfitting.svg.png" />
</center>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Modélisation

In [ ]:
tree = DecisionTreeClassifier(max_depth=6)
tree.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

print("Train :", accuracy_score(y_train, tree.predict(X_train)))
print("Test :", accuracy_score(y_test, tree.predict(X_test)))

In [ ]:
features_imp = pd.DataFrame(
    data=np.asarray([X.columns, tree.feature_importances_]).transpose(),
    columns=['Variable', 'Importance'])
features_imp

In [ ]:
features_imp.set_index("Variable").sort_values(by="Importance").plot.barh(figsize=(14, 9))
for item in ([plt.gca().title, plt.gca().xaxis.label, plt.gca().yaxis.label] +
             plt.gca().get_xticklabels() + plt.gca().get_yticklabels()):
    item.set_fontsize(13)

In [ ]:
import graphviz 
import sklearn.tree

dot_data = sklearn.tree.export_graphviz(tree, out_file=None) 
graph = graphviz.Source(dot_data)
graph